[View in Colaboratory](https://colab.research.google.com/github/shalundia/Kaggle/blob/master/digit-recognizer/mnist_data_CNN.ipynb)

In [6]:
!git clone "https://github.com/shalundia/Kaggle"

Cloning into 'Kaggle'...
remote: Counting objects: 87, done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 87 (delta 42), reused 63 (delta 24), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [8]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs,steps_per_epoch=2000
                              )
'''model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))'''

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/12
2000/2000 [==============================] - 111s 55ms/step - loss: 0.2536 - acc: 0.9217
Epoch 2/12
2000/2000 [==============================] - 111s 55ms/step - loss: 0.1098 - acc: 0.9677
Epoch 3/12
2000/2000 [==============================] - 111s 55ms/step - loss: 0.0919 - acc: 0.9735
Epoch 4/12
2000/2000 [==============================] - 111s 55ms/step - loss: 0.0841 - acc: 0.9764
Epoch 5/12
2000/2000 [==============================] - 110s 55ms/step - loss: 0.0759 - acc: 0.9781
Epoch 6/12
2000/2000 [==============================] - 111s 56ms/step - loss: 0.0727 - acc: 0.9794
Epoch 7/12
2000/2000 [==============================] - 109s 54ms/step - loss: 0.0702 - acc: 0.9805
Epoch 8/12
2000/2000 [==============================] - 106s 53ms/step - loss: 0.0671 - acc: 0.9814
Epoch 9/12
2000/2000 [==============================] - 106s 53ms/step - loss: 0.0676 - acc: 0.9815
Epoch 10/12
2000/2000 [====

In [0]:
import numpy as np
import pandas as pd 

test=np.loadtxt(open('Kaggle/digit-recognizer/test.csv', "rb"), delimiter=",", skiprows=1)


if K.image_data_format() == 'channels_first':
    test = test.reshape(test.shape[0], 1, img_rows, img_cols)
else:
    test = test.reshape(test.shape[0], img_rows, img_cols, 1)

test = test.astype('float32')
test /= 255

prediction=model.predict(test,batch_size=batch_size)

submission=np.argmax(prediction,axis=1)

sub=pd.DataFrame()
sub['ImageId']=np.arange(start=1,stop=(len(test)+1))
sub['Label']=submission
sub.to_csv("submission.csv",index=False)


from google.colab import files
files.download('submission.csv')